## Pre-processing for healthcare analytics

### Load Libraries

In [1]:
import pandas as pd
import os
from helpers6 import *
from past_icu_count import *
from sklearn.impute import SimpleImputer

### Load dataset

In [2]:
#Just place the dataset in the same folder as the notebook
df = pd.read_csv('C:\\Users\\kkb74\\Documents\\SMU Study\\ISSS623 - HealthCare Analytics\\Project\\Group Project Data\master_dataset_2.csv')
df.shape

(448972, 120)

### General Preprocessing

#### Filter out missing age and gender records

In [3]:
# filter age or gender is NaN (837 records)
df = df[~df['age'].isna()]
df = df[~df['gender'].isna()]
df.shape

(448135, 120)

#### Filter out missing acuity (disabled on requested by YX)

In [4]:
# # filter out triage_acuity with null values
df = df[df['triage_acuity'].notnull()]
df.shape

(440786, 120)

#### Corrections to past 30/90/365d ICU stays from ED

In [5]:
df = count_n_days(df, 30)
df = count_n_days(df, 90)
df = count_n_days(df, 365)
df.shape
# no more duplicates found in this version, past ICU stays mapped using index (unique)

C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\past_icu_count.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icu.intime_ed = pd.to_datetime(df_icu.intime_ed) # convert intime to datetime type
C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\past_icu_count.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icu.intime_icu = pd.to_datetime(df_icu.intime_icu) # convert intime to datetime type
C:\Users\kkb74\Doc

(440786, 120)

#### Filter out patients who died before entering ed

In [6]:
# filter out rows where before_ed_mortality is 1 (18 records)
df = df[df['before_ed_mortality'] != 1]
df.shape

(440776, 120)

#### Filter out patients who died during ed

In [7]:
# filter out rows where ed_death is 1 (2 records)
df = df[df['ed_death'] != 1]
df.shape

(440775, 120)

#### Drop duplicate hospital entries on different ed entries

In [8]:
# filter rows where 'hadm_id' is null
df_null = df[df['hadm_id'].isnull()]

# filter rows where 'hadm_id' is not null
df_not_null = df[df['hadm_id'].notnull()]

# Use drop_duplicates to keep the latest occurrence of each 'hadm_id'
df_result = df_not_null.drop_duplicates(subset=['hadm_id'], keep='last')

# Combine the original DataFrame with df_not_null and the duplicates-filtered DataFrame
df = pd.concat([df_null, df_result], ignore_index=True)

print(df_null.shape, "\n", df_not_null.shape, "\n", df_result.shape, "\n", df.shape)

(232131, 120) 
 (208644, 120) 
 (208081, 120) 
 (440212, 120)


#### Drop errorneous entries for time_to_icu_transfer_hours

In [9]:
# filter rows where 'time_to_icu_transfer_hours' is null
df_null = df[df['time_to_icu_transfer_hours'].isnull()]

# filter rows where 'time_to_icu_transfer_hours' is not null
df_not_null = df[df['time_to_icu_transfer_hours'].notnull()]

# Filter the DataFrame where time_to_icu_transfer_hours >= -24
df_result = df_not_null[df_not_null['time_to_icu_transfer_hours'] >= -24]

# Combine the original DataFrame with df_not_null and the duplicates-filtered DataFrame
df = pd.concat([df_null, df_result], ignore_index=True)

df.shape


(440202, 120)

#### Drop errorneous entries for difference between 'admittime' and 'intime_ed'

In [10]:
# Convert 'admittime' and 'intime_ed' to datetime format
df['admittime'] = pd.to_datetime(df['admittime'])
df['intime_ed'] = pd.to_datetime(df['intime_ed'])

# filter rows where 'admittime' or 'admittime' is null
df_null = df[df['admittime'].isnull() | df['intime_ed'].isnull()]

# filter rows where 'admittime' or 'admittime' is not null
df_not_null = df[df['admittime'].notnull() & df['intime_ed'].notnull()]

# Filter rows where the difference between 'admittime' and 'intime_ed' >= -1 hour
df_result = df_not_null[(df_not_null['admittime'] - df_not_null['intime_ed']).dt.total_seconds() > (-3600*24)]

# Combine the original DataFrame with df_not_null and the duplicates-filtered DataFrame
df = pd.concat([df_null, df_result], ignore_index=True)

df.shape

(440144, 120)

#### Drop Features that are not used

In [11]:
# drop features deemeed unnecessary
df = df.drop(['index', 'subject_id', 'hadm_id', 'stay_id', 'anchor_age', 'anchor_year', 'dod',
              'admittime', 'dischtime', 'deathtime', 'race', 'edregtime', 'edouttime', 'in_year',
              'before_ed_mortality', 'ed_los', 'intime_icu', 'outtime_icu', 'time_to_icu_transfer', 'time_to_icu_transfer_hours',
              'next_ed_visit_time', 'next_ed_visit_time_diff', 'next_ed_visit_time_diff_days', 'chiefcomplaint'], axis=1)

df.shape

(440144, 96)

### 2. Outlier Detection

In [12]:
# from mimic-extract
vitals_valid_range = {
    'temperature': {'outlier_low': 14.2, 'valid_low': 26, 'valid_high': 45, 'outlier_high':47},
    'heartrate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 350, 'outlier_high':390},
    'resprate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 300, 'outlier_high':330},
    'o2sat': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 100, 'outlier_high':150},
    'sbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'dbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'pain': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 10, 'outlier_high':10},
    'acuity': {'outlier_low': 1, 'valid_low': 1, 'valid_high': 5, 'outlier_high':5},
}

In [13]:
df = convert_temp_to_celcius(df)

In [14]:
display_outliers_count(df, vitals_valid_range)

C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\helpers6.py:446: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\helpers6.py:446: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\helpers6.py:446: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\helpers6.py:446: Fut

,variable,< outlier_low,"[outlier_low, valid_low)","[valid_low, valid_high]","(valid_high, outlier_high]",> outlier_high
0,triage_temperature,512,29,422137,0,10
1,triage_heartrate,0,0,429299,0,7
2,triage_resprate,0,0,425891,0,4
3,triage_o2sat,0,0,425571,6,43
4,triage_sbp,0,0,428025,0,19
5,triage_dbp,0,0,426782,0,421
6,triage_pain,2,0,405635,0,11823
7,triage_acuity,0,0,440144,0,0
8,ed_temperature_last,670,43,416157,1,48
9,ed_heartrate_last,0,0,424571,0,0


In [15]:
df.shape

(440144, 96)

In [16]:
df.isna().sum()[df.isna().sum() > 0]

insurance              232285
triage_temperature      17456
triage_heartrate        10838
triage_resprate         14249
triage_o2sat            14524
triage_sbp              12100
triage_dbp              12941
triage_pain             22684
ed_temperature_last     23225
ed_heartrate_last       15573
ed_resprate_last        16076
ed_o2sat_last           26732
ed_sbp_last             15810
ed_dbp_last             15810
ed_pain_last            47177
dtype: int64

In [17]:
df = remove_outliers(df, vitals_valid_range)
# what this really does is to transform values between valid low and outlier low to valid low,
# and values between valid high and outlier high to valid high
# anything beyond outlier low and outlier high will be set to NaN
# however the code later imputes NaN with median. i have some concerns with this and would prefer to drop the rows with NaN
# (it's approximately 12k rows only anyway)
df.shape

(440144, 96)

In [18]:
display_outliers_count(df, vitals_valid_range)

C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\helpers6.py:446: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\helpers6.py:446: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\helpers6.py:446: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\kkb74\Documents\SMU Study\ISSS623 - HealthCare Analytics\Project\Project Notebooks\helpers6.py:446: Fut

,variable,< outlier_low,"[outlier_low, valid_low)","[valid_low, valid_high]","(valid_high, outlier_high]",> outlier_high
0,triage_temperature,0,0,422166,0,0
1,triage_heartrate,0,0,429299,0,0
2,triage_resprate,0,0,425891,0,0
3,triage_o2sat,0,0,425577,0,0
4,triage_sbp,0,0,428025,0,0
5,triage_dbp,0,0,426782,0,0
6,triage_pain,0,0,405635,0,0
7,triage_acuity,0,0,440144,0,0
8,ed_temperature_last,0,0,416201,0,0
9,ed_heartrate_last,0,0,424571,0,0


In [19]:
df.isna().sum()[df.isna().sum() > 0]

insurance              232285
triage_temperature      17978
triage_heartrate        10845
triage_resprate         14253
triage_o2sat            14567
triage_sbp              12119
triage_dbp              13362
triage_pain             34509
ed_temperature_last     23943
ed_heartrate_last       15573
ed_resprate_last        16076
ed_o2sat_last           26743
ed_sbp_last             15811
ed_dbp_last             16084
ed_pain_last            47193
dtype: int64

### 3. Data Manipulation for ML

In [20]:
# Change True and False to Binary 1 and 0
df = df.replace(True, 1)
df = df.replace(False, 0)

In [21]:
# Use the replace() method to convert 'M' to 1 and 'F' to 0 in the 'gender' column
df['gender'] = df['gender'].replace({'M': 1, 'F': 0})

In [22]:
# Define the bins and labels for age groups
# Roughly based on NIH (US) age categories, however there is no concensus on ages above 65 years as stated on NIH website
# Age Groups:
# Young Adults: 18 to 34 years
# Middle-Aged Adults: 35 to 64 years
# Older Adults: 65 to 74 years
# Elderly: 76 years and above
bins = [18, 35, 65, 75, 100]  # The age group boundaries
labels = ['Young Adults', 'Middle-Aged Adults', 'Older Adults', 'Elderly']

# Use pd.cut() to bin the 'age' column and replace values with labels
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

#### One-Hot Encodes

In [23]:
# one-hot encode Insurance Type
df = pd.get_dummies(df, columns=['insurance'])

# one-hot encode Age
df = pd.get_dummies(df, columns=['age_group'])

df.shape

(440144, 102)

In [24]:
# Check for Null Values
df.isna().sum()[df.isna().sum() > 0]

triage_temperature     17978
triage_heartrate       10845
triage_resprate        14253
triage_o2sat           14567
triage_sbp             12119
triage_dbp             13362
triage_pain            34509
ed_temperature_last    23943
ed_heartrate_last      15573
ed_resprate_last       16076
ed_o2sat_last          26743
ed_sbp_last            15811
ed_dbp_last            16084
ed_pain_last           47193
dtype: int64

In [25]:
# This code is to convert intime_ed from object data type to datetime data type
df['outtime_ed'] = pd.to_datetime(df['outtime_ed'])


In [26]:
# This code chunk is meant to check the data types of the columns in the data set and save them to a text file for reference.

data_types = df.dtypes

# Convert the data types to a DataFrame for better formatting
data_types_df = pd.DataFrame(data_types, columns=['Data Type'])

# Convert the DataFrame to a string
data_types_str = data_types_df.to_string()

# Save the string to a text file
with open('data_typesv3.txt', 'w') as file:
    file.write(data_types_str)

### 6. Option A: Split Train (80%) and Test (20%) data

In [27]:
# Split Train and Test data
df_train_impute=df.sample(frac=0.8,random_state=2023) #random state is a seed value
df_test_impute=df.drop(df_train_impute.index)
print(df_train_impute.shape)
print(df_test_impute.shape)

(352115, 102)
(88029, 102)


### 4. Option A: Impute NaNs with Median

In [28]:
vitals_cols = [col for col in df.columns if len(col.split('_')) > 1 and 
                                                   col.split('_')[1] in vitals_valid_range and
                                                   col.split('_')[1] != 'acuity']
vitals_cols

['triage_temperature',
 'triage_heartrate',
 'triage_resprate',
 'triage_o2sat',
 'triage_sbp',
 'triage_dbp',
 'triage_pain',
 'ed_temperature_last',
 'ed_heartrate_last',
 'ed_resprate_last',
 'ed_o2sat_last',
 'ed_sbp_last',
 'ed_dbp_last',
 'ed_pain_last']

In [30]:
imputer = SimpleImputer(strategy='median')
df_train_impute[vitals_cols] = imputer.fit_transform(df_train_impute[vitals_cols])
df_test_impute[vitals_cols] = imputer.transform(df_test_impute[vitals_cols])

In [ ]:
# Replace imputation strategy with KNN
# from sklearn.impute import KNNImputer

# # setthe number of neighbors to 5
# imputer = KNNImputer(n_neighbors=5)  

# df_train_impute[vitals_cols] = imputer.fit_transform(df_train_impute[vitals_cols])
# df_test_impute[vitals_cols] = imputer.transform(df_test_impute[vitals_cols])

### 5. Option A: Add Score values to Train data

In [31]:
add_triage_MAP(df_train_impute) # add an extra variable MAP
add_score_CCI(df_train_impute)
add_score_CART(df_train_impute)
add_score_REMS(df_train_impute)
add_score_NEWS(df_train_impute)
add_score_NEWS2(df_train_impute)
add_score_MEWS(df_train_impute)

df_train_impute.shape

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


(352115, 109)

In [32]:
df_train_impute.isna().sum()[df_train_impute.isna().sum() > 0]

Series([], dtype: int64)

### 5. Option A: Add Score values to Test data

In [33]:
add_triage_MAP(df_test_impute) # add an extra variable MAP
add_score_CCI(df_test_impute)
add_score_CART(df_test_impute)
add_score_REMS(df_test_impute)
add_score_NEWS(df_test_impute)
add_score_NEWS2(df_test_impute)
add_score_MEWS(df_test_impute)

df_test_impute.shape

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


(88029, 109)

### 7. Option A: Output the train and test data

In [34]:
df_train_impute.to_csv('C:\\Users\\kkb74\\Documents\\SMU Study\\ISSS623 - HealthCare Analytics\\Project\\Group Project Data\\train_impute_v3a.csv', index=False)
df_test_impute.to_csv('C:\\Users\\kkb74\\Documents\\SMU Study\\ISSS623 - HealthCare Analytics\\Project\\Group Project Data\\test_impute_v3a.csv', index=False)